# Optical flow, motion tracking, segmentation, stereo vision
Erik Matovič  
A solution inspired by [Open CV optical flow tutorial](https://docs.opencv.org/4.x/d4/dee/tutorial_optical_flow.html)   


In [147]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
#from utils import show_img, resize_img, calc_histogram_show, plt_img, equalize_hist, gamma_coorection

print(cv2.__version__)

4.7.0


## Sparse optical flow

Visualize trajectories of moving objects.

Optional task: Identify each object using a bounding box and count them.

Use following functions: cv::goodFeaturesToTrack, cv::calcOpticalFlowPyrLK

In [164]:
def sparse_optical_flow(cap: cv2.VideoCapture, out: cv2.VideoWriter, 
                        ShiTomasi_params: dict, pyrLK_params: dict) -> None:
    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    # ShiTomasi corner detection
    corners = cv2.goodFeaturesToTrack(old_gray, **ShiTomasi_params)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    # list of random colors
    color = np.random.randint(0, 255, (100, 3))

    # Lucas-Kanade Optical Flow
    ret, frame = cap.read()
    while(ret):
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # calculate optical flow
        # nextPts = 2D next points
        # st = status vector, 1 if the the corresponding features has been found
        nextPts, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, corners, None, **pyrLK_params)
        
        # Select good points based on status
        if nextPts is not None:
            good_new = nextPts[st==1]
            good_old = corners[st==1]

        # draw the tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            if i >= len(color):
                  i %= 10 

            a, b = new.ravel()
            c, d = old.ravel()
            pt1, pt2 = (int(a), int(b)), (int(c), int(d))
            mask = cv2.line(mask, pt1, pt2, color[i].tolist()) #, thickness=5)
            #frame = cv2.circle(frame, pt1, 10, color[i].tolist(), -1)
        img = cv2.add(frame, mask)
        #cv2.imshow('frame', img)
        #cv2.waitKey(0)
        # write the flipped frame
        out.write(img)
        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        corners = good_new.reshape(-1, 1, 2)

        # read next frame
        ret, frame = cap.read()
        
    # Release everything if job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [165]:
def get_cap_out(video_path:str) -> tuple:
    """
    returns: cv2.VideoCapture, cv2.VideoWriter 
    """
    # load video
    cap = cv2.VideoCapture(video_path)

    # Default resolutions of the frame are obtained.The default resolutions are system dependent.
    # We convert the resolutions from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))

    # make video writer
    out = cv2.VideoWriter('..' + video_path[15:-4] + '.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))
    return cap, out

In [166]:
ShiTomasi_params = dict(mask = None, 
    maxCorners = 10000,         # max number of corners to return
    qualityLevel = 0.1,       # min accepted quality of img corners
    minDistance = 1,           # min possible Euclidean distance between the returned corners
    blockSize = 1,             # size of an average block for computing a derivative covariation matrix over each pixel neighborhood
    useHarrisDetector = False 
)

pyrLK_params = dict(winSize  = (41, 41),   # size of the search window at each pyramid lvl
                 maxLevel = 4,          # max pyramid level number
                 # termination criteria
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

cap, out = get_cap_out('../data/archive/fourway.avi')
sparse_optical_flow(cap, out, ShiTomasi_params, pyrLK_params)

In [167]:
ShiTomasi_params = dict(mask = None, 
    maxCorners = 10000,         # max number of corners to return
    qualityLevel = 0.1,       # min accepted quality of img corners
    minDistance = 1,           # min possible Euclidean distance between the returned corners
    blockSize = 1,             # size of an average block for computing a derivative covariation matrix over each pixel neighborhood
    useHarrisDetector = False 
)

pyrLK_params = dict(winSize  = (41, 41),   # size of the search window at each pyramid lvl
                 maxLevel = 4,          # max pyramid level number
                 # termination criteria
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
cap, out = get_cap_out('../data/archive/crosswalk.avi')
sparse_optical_flow(cap, out, ShiTomasi_params, pyrLK_params)

In [168]:
ShiTomasi_params = dict(mask = None, 
    maxCorners = 10000,         # max number of corners to return
    qualityLevel = 0.1,       # min accepted quality of img corners
    minDistance = 1,           # min possible Euclidean distance between the returned corners
    blockSize = 1,             # size of an average block for computing a derivative covariation matrix over each pixel neighborhood
    useHarrisDetector = False 
)

pyrLK_params = dict(winSize  = (41, 41),   # size of the search window at each pyramid lvl
                 maxLevel = 4,          # max pyramid level number
                 # termination criteria
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

cap, out = get_cap_out('../data/archive/night.avi')
sparse_optical_flow(cap, out, ShiTomasi_params, pyrLK_params)

## Dense optical flow

Identify moving objects in video and draw green rectangle around them.

Use downsampled video for this task if necessary for easier processing.

Use following functions: cv::calcOpticalFlowFarneback

[OpenCV's tutorial on how to optical flow](https://docs.opencv.org/4.x/d4/dee/tutorial_optical_flow.html)

Motion tracking Datasets

Feel free to experiment with multiple videos for motion tracking. Use the following link for additional datasets - https://motchallenge.net/data/MOT15/

## Segmentation using background subtraction

Use background substraction methods to properly segment the moving objects from their background. Use one of the videos with static camera.

Use the following approaches:

    Accumulated weighted image

    Mixture of Gaussian (MOG2)

## Grab Cut segmentation

Propose a simple method to segment a rough estimate of lateral ventricle segmentation using morphological processing and thresholding.

[Link, 5 x PNG, 137 KB](https://drive.google.com/file/d/1hnQ_PHx0LhMNCMlpwCFhXVx4fFl9j_Aq/view) 

Use OpenCV's graph cut method to refine segmentation boundary.

cv::grabCut

Input has to be BGR (3 channel)

Values for the mask parameter:

GC_BGD = 0 - an obvious background pixels

GC_FGD = 1 - an obvious foreground (object) pixel

GC_PR_BGD = 2 - a possible background pixel

GC_PR_FGD = 3  - a possible foreground pixel

An example of GrabCut algorithm: link (note: This example uses a defined rectangle for grabcut segmentation. In our case we want to use the mask option instead)